In [1]:
# Importando bibliotecas utilizadas
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from math import sqrt, pow
from datetime import datetime
from time import sleep

In [2]:
from scipy.spatial.distance import pdist, squareform

In [3]:
df = pd.read_csv('datasets/c2ds1-2sp.txt', sep='\t')
df.head()

,sample_label,d1,d2
0,c2sp1s1,10.500000,9.000000
1,c2sp1s2,10.567170,9.268445
2,c2sp1s3,8.275320,11.382210
3,c2sp1s4,8.227458,11.377640
4,c2sp1s5,8.179511,11.372110


In [4]:
df3 = pd.read_csv('datasets/monkey.txt', sep='\t')
df3.columns = ['identificador','d1', 'd2']
df3.head()

,identificador,d1,d2
0,monkeyc1g1s1,8.809783,7.611147
1,monkeyc1g1s2,4.110747,11.103186
2,monkeyc1g1s3,4.114710,11.039587
3,monkeyc1g1s4,3.154736,6.743244
4,monkeyc1g1s5,5.972931,7.537982


# Algoritmo

In [5]:
#Distancia Euclidiana
# Mudei a forma como é calculado para poder utilizar vetorização
def distancia_euclidiana(x1, y1, x2, y2):
    X = x1 - x2
    Y = y1 - y2
    
    X = X*X
    Y = Y*Y
    
    distancia = np.sqrt(X+Y)
    return distancia

In [6]:
def calcular_distancia_full_optimized(df):
    lista = []
    for i, row in df.iterrows():
        distancias = distancia_euclidiana(row['d1'], row['d2'], df[i+1:]['d1'].values, df[i+1:]['d2'].values)
        
        lista.append(distancias)
      
    return lista

In [7]:
def dist_matrix(df):
    lista = []
    for i, row in df.iterrows():
        distancias = distancia_euclidiana(row['d1'], row['d2'], df['d1'].values, df['d2'].values)

        lista.append(distancias)
    
    matrix = np.matrix(lista)
    np.fill_diagonal(matrix, float('inf'))
    return matrix

In [ ]:
def get_pos(u, v, num_obj):
    return u*num_obj + v

In [ ]:
def criar_lista_de_indices(pos_menor, pos_maior, numero_de_objetos):
    lista = np.array([], dtype=int)
    for i in pos_menor:
        aux = get_pos(i, pos_maior, numero_de_objetos)
        lista = np.concatenate((lista,aux))
        
    return lista

In [ ]:
def agrupar_clusters(matriz, clusters, objetos, numero_de_objetos):
    obj1, obj2 =  objetos
    cluster1 = clusters[obj1]
    cluster2 = clusters[obj2]
    
    if cluster1 == cluster2:
        return False
    
    if cluster1 < cluster2:
        menor_cluster = cluster1
        maior_cluster = cluster2
    else:
        menor_cluster = cluster2
        maior_cluster = cluster1
        
    pos_menor = np.array(np.where(clusters == menor_cluster))
    pos_maior = np.array(np.where(clusters == maior_cluster))
    
#     print(f"Número de objetos {numero_de_objetos}")
#     print(f"Posição cluster menor {pos_menor}")
#     print(f"Posição cluster maior {pos_maior}\n\n")
#     print(lista_de_indices_inf)
    lista_de_indices_inf = criar_lista_de_indices(pos_menor[0], pos_maior[0], numero_de_objetos)
#     print(f'Lista de índices {lista_de_indices_inf}')
    
    np.put(matriz, lista_de_indices_inf, float('inf'))
    
    clusters[clusters == maior_cluster] = menor_cluster
    return True

In [ ]:
def single_link(dataset, k_min, k_max, nome):
    numero_de_objetos = len(dataset)
    matrix = dist_matrix(dataset)
    clusters = np.arange(numero_de_objetos)
    
    qtd_clusters = numero_de_objetos
    while qtd_clusters > k_min:
        objetos = matrix.argmin()
        i = objetos // numero_de_objetos
        j = objetos % numero_de_objetos
        matrix[i,j] = float('inf')
        matrix[j,i] = float('inf')
        
        if agrupar_clusters(matrix, clusters, [i,j], numero_de_objetos):
            if qtd_clusters % 100 == 0:
                print(qtd_clusters)
            qtd_clusters -= 1
    
    return clusters

# MST

Após os testes com vetorização ainda ficou muito lento, próximos dos 30min, então fui ler sobre e descobri que a forma mais otimizada do Single Link utiliza algo semelhante ao algoritmo de Prim.

[Link do artigo](http://eprints.hud.ac.uk/id/eprint/32552/1/DWD_Fmurtagh_31.pdf)
[NN-chain](https://en.wikipedia.org/wiki/Nearest-neighbor_chain_algorithm)
[Prim's Algorithm)https://en.wikipedia.org/wiki/Prim%27s_algorithm

# Testes 

In [ ]:
df_menor = df[:10].copy()
df_menor

In [ ]:
clusters = single_link(df_menor, 2, 5, 'teste')

In [ ]:
teste1 = distancia_euclidiana(10.5,9, df_menor['d1'], df_menor['d2'])
teste1

In [ ]:
condensed_matrix = pdist(df_menor[['d1', 'd2']])
condensed_matrix

In [ ]:
converted_condensed_matrix = squareform(pdist(df_menor[['d1', 'd2']]))
converted_condensed_matrix

In [ ]:
inicio = datetime.now()
clusters = single_link(df3, 5, 10, 'teste')
fim = datetime.now()

## Difrença entre `calcular_distancia_full_optimized` x  `dist_matrix`
Testes sobre a diferença de tempo entre calcular a matriz completa e apenas metade dela

In [ ]:
%%timeit
teste = dist_matrix(df3)


In [ ]:
%%timeit

teste = calcular_distancia_full_optimized(df3)

O ganho de calcular a matriz inteira não é muito relevante, mas temos que considerar que ao calcular a matriz inteira ganhamos um leque muito grande de funções do ```numpy``` que devem ser adicionados as contas ao alterar a implementeção do algoritmo em si

## Colocando inf nas distâncias entre os mesmos clusters

Eu acho que a parte mais está demorando no momento é quando ele pega distâncias que pertencem ao mesmo cluster e com isso tem que realizar várias iterações até achar uma que funcione. Por isso o que vou tentar fazer é já colocar `float('inf')` nas distâncias de mesmo cluster quando ele juntar.

In [ ]:
matriz = dist_matrix(df_menor[:5])
clusters = np.arange(10)

matriz

In [ ]:
lista = np.array([0,5,2])
np.put(matriz, lista, float('inf'))
matriz

In [ ]:
vet1 = np.array([1,2,3,4])
vet2 = np.array((0,0,5,3))

def get_pos(u, v, num_obj):
    return u*num_obj + v


teste = get_pos(vet1, vet2, 5)
teste = list(teste)
teste

In [ ]:
np.put(matriz, teste, float('inf'))
matriz

In [ ]:
def agrupar_clusters_teste(matriz, clusters, objetos, numero_de_objetos):
    obj1, obj2 =  objetos
    cluster1 = clusters[obj1]
    cluster2 = clusters[obj2]
    
    if cluster1 == cluster2:
        return False
    
    if cluster1 < cluster2:
        menor_cluster = cluster1
        maior_cluster = cluster2
    else:
        menor_cluster = cluster2
        maior_cluster = cluster1
        
    pos_menor = np.array(np.where(clusters == menor_cluster))
    pos_maior = np.array(np.where(clusters == maior_cluster))
    
#     print(f"Número de objetos {numero_de_objetos}")
#     print(f"Posição cluster menor {pos_menor}")
#     print(f"Posição cluster maior {pos_maior}\n\n")
#     print(lista_de_indices_inf)
    lista_de_indices_inf = criar_lista_de_indices(pos_menor[0], pos_maior[0], numero_de_objetos)
    print(f'Lista de índices {lista_de_indices_inf}')
    
    np.put(matriz, lista_de_indices_inf, float('inf'))
    
    clusters[clusters == maior_cluster] = menor_cluster
    return True

In [ ]:
matriz = np.zeros((8,8))
clusters = np.array([0,0,0,5,5,5,2,2], dtype=float)
numero_de_objetos = len(clusters)
obj1 = 5
obj2 = 2

agrupar_clusters_teste(matriz, clusters, [obj1,obj2], numero_de_objetos)

In [ ]:
matriz